# Example GATIS Conversion Pipeline with Existing Active Transportation Data from Newark, Delaware 
---


This notebook demonstrates a potential workflow for converting existing active transportation data into GATIS format. Data are converted into GATIS format using our best guess at what the mapping would be, so there may be certain attributes that were misinterpreted or missed. In terms of GATIS' tier structure, this data is Tier 2, though it contains some additional attributes above Tier 2.

The final sample data can be accessed using the following links:
<ul>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=936f374afa8d4612ba1fafaf3eedc000'>Sample GATIS Edges (Austin, TX) Feature Layer</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=bd0e32d7b09e4cd6a9cb41a6ac8b8864'>Sample GATIS Edges (Austin, TX) GeoJSON Download</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=1c563a62a83244ba9edbeaa6eafee6a2'>Sample GATIS Nodes (Austin, TX) Feature Layer</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=5b63f3dd5adf4503a756c2c0e9738a2c'>Sample GATIS Nodes (Austin, TX) GeoJSON Download</a></li>
</ul>

This notebook can be run using Jupyter Notebooks with a Python environment with the latest version of GeoPandas installed. The outputs of this notebook are retained in this file for reference.

Feel free to create a copy of this notebook and modify as needed to convert other data sources to GATIS.

## Data Sources
The table below provides links to the various datasets used to create the sample GATIS data. To re-run this notebook these data needs to be downloaded and placed in a folder called `data` within the same directory as this notebook.

| Filename | Description |
| -- | -- |
| [Sidewalks_20250825.geojson](https://data.austintexas.gov/dataset/Sidewalks/vchz-d9ng/about_data) | Latest and most complete version of Austin sidewalks |
| [TRANSPORTATION.markings_short_line_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION-markings_short_line/9hak-anfp/about_data) | Only kept crosswalks |
| [TRANSPORTATION_curb_ramps.geojson](https://hub.arcgis.com/datasets/a477eed132a04d26b70e74d8fba74c42_0/explore?location=30.322658%2C-97.746550%2C10.83&uiVersion=content-views) | Has curb ramps |
| [TRANSPORTATION_asmp_street_network_20250825.geojson](https://data.austintexas.gov/Locations-and-Maps/TRANSPORTATION_asmp_street_network/cr24-n8br/about_data) | This one appears to conflict partially with the bicycle facilities one but contains more streets |
| [TRANSPORTATION_urban_trails_network_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_urban_trails_network/jdwm-wfps/about_data) | Centerlines for bikeped trails that aren't in the other files |

<!-- These were not included:
| [Completed_Sidewalks_2021_09_30_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Completed_Sidewalks_2021_09_30/6pkw-7frd/about_data) | Used Sidewalks_20250825.geojson instead |
| [Flashing Beacons_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Flashing-Beacons/wczq-5cer/about_data) | Contains PBHs in point format, would need to be spatially joined to crosswalk edges to be compliant with GATIS |
| [Roadway Sign Assets_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Roadway-Sign-Assets/ggnk-3ykn/about_data) | Signs haven't made it into GATIS yet |
| [Traffic Signals and Pedestrian Signals_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Traffic-Signals-and-Pedestrian-Signals/p53x-x73x/about_data) | Needs to be spatially joined to the crosswalk edges |
| [TRANSPORTATION_bicycle_facilities_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_bicycle_facilities/23hw-a95n/about_data) | Overlaps with asmp_street_network and urban_trails |
| [TRANSPORTATION_markings_long_line_20250825.csv](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_markings_long_line/nxkm-x3yr/about_data) | Didn't have a geometry column, might need to be joined to one of the other datasets? Does have bike facilities. |
| [TRANSPORTATION_markings_specialty_line_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_markings_specialty_line/rczx-h2ey/about_data) | Contains some bike facility separation components that could be joined to bike facilities |
| [TRANSPORTATION_markings_specialty_point_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_markings_specialty_point/53jy-y8pj/about_data) | Points for thermoplastic/paint pictogram markings on the road. Not applicable to GATIS. | -->

## Import required packages

In [5]:
import requests
import numpy as np
import geopandas as gpd
from shapely.ops import Point
from shapely.geometry import MultiLineString, Point

bbox = None

In [6]:
gdb_path = "/Users/Nineveh.OConnell/OneDrive - DOT OST/Documents/NCBPAID_Newark_Bike.gdb"
# primary 
layer_name = "LTSv4_Network_Project_Newark_Intersect"

gdf = gpd.read_file(gdb_path, layer = layer_name, driver='OpenFileGDB', mask = bbox).explode()

c:\Users\Nineveh.OConnell\AppData\Local\anaconda3\envs\BPA\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
c:\Users\Nineveh.OConnell\AppData\Local\anaconda3\envs\BPA\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiLineString' is converted to 'MultiLineString Z'
  return ogr_read(


## GATIS Template

The GATIS template can be used to create a DataFrame/Table of the required/recommended fields at any particular Tier level. If desired, you can also decide to include optional fields. For this demonstration, we'll bring in the Tier 4 template so that we have all fields.

In [7]:
# import tier 4 GATIS template so that all of the possible attributes are in there
gatis_template_url = "https://raw.githubusercontent.com/dotbts/BPA/main/draft_gatis_specification/templates/tier_4_and_optional"
gatis_edges_geojson = requests.get(gatis_template_url + "/edges.geojson").json()
gatis_nodes_geojson = requests.get(gatis_template_url + "/nodes.geojson").json()

In [8]:
# changing fields to GATIS structure in place, will drop old columns once it's all set up

# edge id, take the existing unique id
gdf['edge_id'] = gdf['v4_FID']

# replace edge types with a reasonable association of road or multi use path
edge_type_map = {'LRS_Inventory':'road', 'NonLRS_Residential':'road', 
                 'Pathways_Park':'multi_use_path', 'NonLRS_Service':'road', 
                 'NonLRS_Tertiary':'road', 'NonLRS_ParkingLot1':'road',
                 'NonLRS_Divided':'road', 'NonLRS_Functional':'road', 
                 'Pathways_Sidepath':'multi_use_path', 'NonLRS_ParkingLot2':'road', 
                 'Pathways_Connector':'multi_use_path', 'Pathways':'multi_use_path',
                 'Pathways_Other':'multi_use_path', 'NonLRS_Aggregation':'road'}
gdf['edge_type'] = gdf['LRS_CODE'].replace(edge_type_map)
gdf['street_name'] = gdf['STREETNAME']

# function to round the two significant figures
def round_to_sig_figs(number, n_sig_figs):
    # Calculate the power of 10 for the most significant digit
    power = 10 ** np.floor(np.log10(np.abs(number)))
    
    # Round the number scaled by this power, then scale it back
    rounded_number = np.round(number / power, n_sig_figs - 1) * power
    return rounded_number

# flag road associated now only for those that are not deemed road
# as road_associated is a forbidden attribute for roads themselves
gdf['road_associated'] = np.where((gdf['ROAD'] == 0), 'no', np.where((gdf['edge_type'] != 'road'), 'yes', None))
gdf['facility_name'] = np.where(gdf['edge_type'] != 'road', gdf['STREETNAME'], None)

# traffic volume -- vehicle traffic annual average daily
gdf['traffic_volume'] = np.where(gdf['edge_type'] == 'road', round_to_sig_figs(gdf['CURRENT_AA'],2), None)

# need to look more into the divided and one ways to confirm they are forward, not backward
# also to remind myself if the divided is each way separately recorded, or if it's all together as one
gdf['mp_diff'] = gdf['END_MP'] - gdf['BEG_MP']
# checked the above, they are all positive so I'm using the below to define directionality
# set multi use paths as both directions as well
gdf['directionality'] = np.where((gdf['ONEWAY'] == 'Bidirectional') | (gdf['edge_type'] == 'multi_use_path'), 'both', 'forward')

# incline -- need to adjust this slope metric to percentage from whatever it is currently
gdf['incline'] = np.abs(gdf['R_slope_avg'])

# posted speed limit, 85th percentile free flow motor vehicle speed
gdf['posted_speed_limit'] = gdf['SPEEDLIMIT']
gdf['car_freeflow_speed'] = np.where(gdf['edge_type'] == 'road', gdf['FL33T85'], None)

# road centerline presence
gdf['roadway_centerline'] = np.where((gdf['edge_type'] == 'road') & (gdf['CNTRLNE'] == 0), 'no', 'yes')

# additionally available attributes, recommended at tier 2
gdf['thru_lanes'] = gdf['LANE_THRU']
gdf['aux_lanes'] = gdf['R_LANE_N'] # this is right only currently. should i sum left, or split out left seperately?
gdf['shoulder_width'] = gdf['R_SHLDR_W'] # this is the right only, same situation as above

# we don't have overall road width provided

c:\Users\Nineveh.OConnell\AppData\Local\anaconda3\envs\BPA\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
# going to make everything centerline at first, and then shift over the column names for non-road edges

# lets do right bike lane, then same for left
gdf['bikeway:right:presence'] = np.where(gdf['R_BL_W'] > 0, 'yes', 'no')
gdf['bikeway:right:width'] = np.where(gdf['R_BL_W'] > 0, 12*gdf['R_BL_W'], None)
gdf['bikeway:right:bikeway_type'] = np.where(gdf['R_BL_T'] == 'Striped/ Parking', 'Separated Bike Lane', np.where(gdf['R_BL_T'] == 'Striped', 'Bike Lane', 'Shared Lane'))
gdf['bikeway:right:separation_elements'] = np.where(gdf['bikeway:right:bikeway_type'] == 'Separated Bike Lane', {'parking'}, {''})
gdf['bikeway:right:separation_permeable_car'] = np.where(gdf['bikeway:right:bikeway_type'] == 'Separated Bike Lane', 'soft separator', 'none')

# all the same for left, with the same questions outstanding
gdf['bikeway:left:presence'] = np.where(gdf['R_BL_W'] > 0, 'yes', 'no')
gdf['bikeway:left:width'] = np.where(gdf['R_BL_W'] > 0, 12*gdf['R_BL_W'], None)
gdf['bikeway:left:bikeway_type'] = np.where(gdf['R_BL_T'] == 'Striped/ Parking', 'Separated Bike Lane', np.where(gdf['R_BL_T'] == 'Striped', 'Bike Lane', 'Shared Lane'))
gdf['bikeway:left:separation_elements'] = np.where(gdf['bikeway:left:bikeway_type'] == 'Separated Bike Lane', 'parking', None) # this should be formatted as lists? how does that work
gdf['bikeway:left:separation_permeable_car'] = np.where(gdf['bikeway:left:bikeway_type'] == 'Separated Bike Lane', 'soft separator', 'none')


In [10]:
# generating additional fields so that the bikeway attributes of the roads are to a tier 3 level

# we'll call it a bridge if bridge is in the street name text
gdf['bridge'] = np.where(gdf['STREETNAME'].str.contains(' bridge', case=False), 'yes', 'no')

# assume all at grade, with the exception of delaware avenue
gdf['bikeway:right:bikeway_grade_separation'] = np.where(gdf['bikeway:right:presence'] == 'yes', np.where(gdf['street_name'].str.contains('delaware ave', case=False), 'sidewalk_level', 'at_grade'), None)
gdf['bikeway:left:bikeway_grade_separation'] = np.where(gdf['bikeway:left:presence'] == 'yes', np.where(gdf['street_name'].str.contains('delaware ave', case=False), 'sidewalk_level', 'at_grade'), None)

# assume minimum width of 5 feet for bikeways, unless the stated width is already narrower
gdf['bikeway:right:width_min'] = np.where((gdf['bikeway:right:presence'] == 'yes') & (gdf['bikeway:right:width'] > 60), 60, gdf['bikeway:right:width'])
gdf['bikeway:left:width_min'] = np.where(gdf['bikeway:left:width'] <= 60, gdf['bikeway:left:width'], 60)

# assume all bike lanes are currently open
gdf['bikeway:right:status'] = 'open'
gdf['bikeway:left:status']  = 'open'

# assume all bikeways on roads are asphalt, all else are concrete
gdf['bikeway:right:surface_material'] = np.where(gdf['edge_type'] =='road', 'asphalt', 'concrete')
gdf['bikeway:left:surface_material'] = np.where(gdf['edge_type'] =='road', 'asphalt', 'concrete')

In [ ]:
# generate nodes...
start_points = []
end_points = []
for linestring in gdf.geometry:
    # Get the first and last coordinate of each LineString
    start_points.append(Point(linestring.coords[0]))
    end_points.append(Point(linestring.coords[-1]))


In [ ]:
#subset to columns in the GATIS spec
gatis_edge_col_names = ['geometry', 'edge_id', 'reference_ids', 'street_name', 'edge_type', 'from_node', 
                        'to_node', 'directionality', 'width', 'width_min', 'width_tolerance', 
                        'traffic_volume', 'posted_speed_limit', 'car_freeflow_speed', 'thru_lanes', 
                        'aux_lanes', 'shoulder_width', 'roadway_centerline', 'bridge', 'prohibited_uses', 
                        'allowed_uses', 'surface_material', 'surface_issue', 'status', 'seasonal', 
                        'incline', 'cross_slope', 'cross_slope_max', 'impediment', 'date_built', 
                        'check_date', 'traffic_calming', 'curb_height', 'detectable_warning', 'measured_length', 
                        'bikeway:left:presence', 'bikeway:left:reference_ids', 'bikeway:left:facility_name', 
                        'bikeway:left:directionality', 'bikeway:left:width', 'bikeway:left:width_min', 
                        'bikeway:left:width_tolerance', 'bikeway:left:bikeway_type', 'bikeway:left:bikeway_grade_separation', 
                        'bikeway:left:separation_elements', 'bikeway:left:separation_permeable_car', 'bikeway:left:buffer_width', 
                        'bikeway:left:street_parking', 'bikeway:left:street_parking_buffer', 'bikeway:left:posted_speed_limit', 
                        'bikeway:left:prohibited_uses', 'bikeway:left:allowed_uses', 'bikeway:left:surface_material', 
                        'bikeway:left:surface_issue', 'bikeway:left:status', 'bikeway:left:seasonal', 'bikeway:left:incline', 
                        'bikeway:left:cross_slope', 'bikeway:left:cross_slope_max', 'bikeway:left:impediment', 'bikeway:left:date_built', 
                        'bikeway:left:check_date', 'bikeway:left:detectable_warning', 'bikeway:left:measured_length', 
                        'bikeway:right:presence', 'bikeway:right:reference_ids', 'bikeway:right:facility_name', 'bikeway:right:directionality', 
                        'bikeway:right:width', 'bikeway:right:width_min', 'bikeway:right:width_tolerance', 'bikeway:right:bikeway_type', 'bikeway:right:bikeway_grade_separation', 
                        'bikeway:right:separation_elements', 'bikeway:right:separation_permeable_car', 'bikeway:right:buffer_width', 'bikeway:right:street_parking', 
                        'bikeway:right:street_parking_buffer', 'bikeway:right:posted_speed_limit', 'bikeway:right:prohibited_uses', 'bikeway:right:allowed_uses', 
                        'bikeway:right:surface_material', 'bikeway:right:surface_issue', 'bikeway:right:status', 'bikeway:right:seasonal', 'bikeway:right:incline', 
                        'bikeway:right:cross_slope', 'bikeway:right:cross_slope_max', 'bikeway:right:impediment', 'bikeway:right:date_built', 'bikeway:right:check_date', 
                        'bikeway:right:detectable_warning', 'bikeway:right:measured_length', 'status', 'road_associated']

In [ ]:
# create reference ids
reference_id = gdf[["edge_id"]].to_dict(orient='records')
gdf['reference_ids'] = [[{'source': 'newark',**item}] for item in reference_id]

# subset to keep only the gatis-ified columns
cols_to_keep = set(gatis_edge_col_names) & set(gdf.columns.tolist())
sub_gdf = gdf[list(cols_to_keep)]

# splitting out the multi use paths into a separate table from the roads
gdf_roads = sub_gdf[sub_gdf['edge_type'] == 'road']
gdf_mup = sub_gdf[sub_gdf['edge_type'] == 'multi_use_path']

# drop bikeway prefixes from column names
# this is still underway, just uploading now
gdf_mup = gdf_mup[['geometry', 'edge_id', 'status', 'road_associated', 'reference_ids', 'street_name', 'edge_type', 'from_node', 
                    'to_node', 'directionality', 'width', 'width_min', 'surface_material', 'surface_issue', 'status', 
                    'incline', 'cross_slope', 'cross_slope_max', 'impediment', 'date_built', 
                    'check_date', 'traffic_calming', 'curb_height', 'detectable_warning', 'measured_length']]




below is holdover from the austin script, keeping it around if useful

In [ ]:
# export a gpkg version for viewing in qgis
gatis_edges_gdf.loc[:,gatis_edges_gdf.isna().all(axis=0)==False].to_file("austin_sample.gpkg",layer="gatis_edges")
gatis_nodes_gdf.loc[:,gatis_nodes_gdf.isna().all(axis=0)==False].to_file("austin_sample.gpkg",layer="gatis_nodes")

# export the GeoJSON version
with open("austin_sample_edges.geojson","w") as f:
    f.write(gatis_edges_gdf.to_json(na="drop",indent=2,drop_id=True,to_wgs84=True))
with open("austin_sample_nodes.geojson","w") as f:
    f.write(gatis_nodes_gdf.to_json(na="drop",indent=2,drop_id=True,to_wgs84=True))

In [ ]:
bboxes = []
bboxes.append(shapely.from_wkt("POLYGON((-97.748244 30.278771, -97.736805 30.278771, -97.736805 30.267221, -97.748244 30.267221, -97.748244 30.278771))"))
bboxes.append(shapely.from_wkt("POLYGON((-97.729976 30.258163, -97.718538 30.258163, -97.718538 30.246611, -97.729976 30.246611, -97.729976 30.258163))"))

# write a description for the data
description = \
"""This sample dataset was created from Austin's existing municipal data of streets, trails, sidewalks, crosswalks, and curb ramps. In terms of GATIS' tier structure, this data is Tier 2, though it contains some additional attributes above Tier 2. Note, this is only a clipped sample of the complete dataset; the full dataset can be downloaded using the following links:

<ul>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=936f374afa8d4612ba1fafaf3eedc000'>Sample GATIS Edges (Austin, TX) Feature Layer</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=bd0e32d7b09e4cd6a9cb41a6ac8b8864'>Sample GATIS Edges (Austin, TX) GeoJSON Download</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=1c563a62a83244ba9edbeaa6eafee6a2'>Sample GATIS Nodes (Austin, TX) Feature Layer</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=5b63f3dd5adf4503a756c2c0e9738a2c'>Sample GATIS Nodes (Austin, TX) GeoJSON Download</a></li>
</ul>

See the full process <a href='https://github.com/dotbts/BPA/blob/main/draft_gatis_specification/sample_data/austin/Austin_GATIS_Conversion.ipynb'>here</a>.
"""

for idx, bbox in enumerate(bboxes):
    if bbox is not None:
        masked_edges = gatis_edges_gdf[gatis_edges_gdf['geometry'].intersects(bbox)]
        masked_nodes = gatis_nodes_gdf[gatis_nodes_gdf['geometry'].intersects(bbox)]
        m = create_maps.display_layers(masked_edges, masked_nodes, edge_categories="edge_type", node_categories="node_type")
        m.save(here(f"draft_gatis_specification/sample_data/maps/austin_{idx+1}.html"))

        maps_dict = {
            "key": f"austin_{idx+1}",
            "label": f"Austin, TX {idx+1} (City of Austin)",
            "description": description
        }
        # saves to maps.json and creates a new key for the path to the map for the GATIS Explorer
        create_maps.add_to_json(maps_dict)
